# FastANI
Summary of [FastANI](https://github.com/ParBLiSS/FastANI) results from project: `[{{ project().name }}]`

## Description
Fast Whole-Genome Similarity (ANI) Estimation

In [ ]:
import pandas as pd
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

#import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.preprocessing import MinMaxScaler
import numpy as np
#import networkx as nx
#import community as community_louvain
sns.set_context("paper")


def kMeansRes(scaled_data, k, alpha_k=0.02):
    '''
    # Calculating clusters from https://medium.com/towards-data-science/an-approach-for-choosing-number-of-clusters-for-k-means-c28e614ecb2c
    Parameters 
    ----------
    scaled_data: matrix 
        scaled data. rows are samples and columns are features for clustering
    k: int
        current k for applying KMeans
    alpha_k: float
        manually tuned factor that gives penalty to the number of clusters
    Returns 
    -------
    scaled_inertia: float
        scaled inertia value for current k           
    '''
    
    inertia_o = np.square((scaled_data - scaled_data.mean(axis=0))).sum()
    # fit k-means
    kmeans = KMeans(n_clusters=k, random_state=0).fit(scaled_data)
    scaled_inertia = kmeans.inertia_ / inertia_o + alpha_k * k
    return scaled_inertia

def chooseBestKforKMeans(scaled_data, k_range):
    ans = []
    for k in k_range:
        scaled_inertia = kMeansRes(scaled_data, k)
        ans.append((k, scaled_inertia))
    results = pd.DataFrame(ans, columns = ['k','Scaled Inertia']).set_index('k')
    best_k = results.idxmin()[0]
    return best_k, results

In [ ]:
report_dir = Path("../")

In [ ]:
df_fastani = pd.read_csv(report_dir / 'fastani/df_fastani.csv', index_col=0)
df_gtdb = pd.read_csv(report_dir / 'tables' / 'df_gtdb_meta.csv', index_col='genome_id')

## Hierarchical Clustering based on ANI values

In [ ]:
df_fastani_corr = df_fastani.fillna(0).corr()

plt.figure(figsize=(30, 7))
plt.title("FastANI Similarity")

selected_data = df_fastani_corr.copy()
clusters = shc.linkage(selected_data, 
            method='ward', 
            metric="euclidean",
            optimal_ordering=True,)
shc.dendrogram(Z=clusters, labels=df_fastani_corr.index, leaf_rotation=90)
plt.show()

## Estimate Number of Clusters

In [ ]:
# choose features
data_for_clustering = df_fastani.copy()
data_for_clustering.fillna(0,inplace=True)

# create data matrix
data_matrix = np.asarray(data_for_clustering).astype(float)
data_matrix

# scale the data
mms = MinMaxScaler()
scaled_data = mms.fit_transform(data_matrix)

# choose k range
if len(df_fastani) <= 21:
    max_range = len(df_fastani) - 1
else:
    max_range = 20

k_range=range(2, max_range)
# compute adjusted intertia
best_k, results = chooseBestKforKMeans(scaled_data, k_range)

# plot the results
plt.figure(figsize=(7,4))
plt.plot(results,'o')
plt.title('Adjusted Inertia for each K')
plt.xlabel('K')
plt.ylabel('Adjusted Inertia')
plt.xticks(range(2,max_range,1))
print(f"Estimated number of clusters: {best_k}")
plt.show()

## ANI Clustermap

In [ ]:
n_clusters = best_k

# max color 12
if best_k < 12:
    top_clusters = best_k
else:
    top_clusters = 12

Agg_hc = AgglomerativeClustering(n_clusters = n_clusters, affinity = 'euclidean', linkage = 'ward')
y_hc = Agg_hc.fit_predict(df_fastani_corr)
color_set3 = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

df_hclusts = pd.DataFrame(index=df_fastani_corr.index, columns=['hcluster', 'color_code'])
df_hclusts['hcluster'] = y_hc
top_clusters = df_hclusts.hcluster.value_counts().index.tolist()[:top_clusters]
dict_top_colors = dict(zip(top_clusters, color_set3[:len(top_clusters)]))

for genome_id in df_hclusts.index:
    cluster_id = df_hclusts.loc[genome_id, 'hcluster']
    if cluster_id in top_clusters:
        df_hclusts.loc[genome_id, 'color_code'] = dict_top_colors[cluster_id]
    else:
        df_hclusts.loc[genome_id, 'color_code'] = "#808080"
        
comm_colors = df_hclusts['color_code']
plt.figure()
# sns.set_theme(color_codes=True)
g = sns.clustermap(df_fastani,
                  figsize=(50,50), row_linkage=clusters, col_linkage=clusters,
                  row_colors=comm_colors, col_colors=comm_colors)
g.ax_heatmap.set_xlabel('Genomes')
g.ax_heatmap.set_ylabel('Genomes')
plt.show()

## References
<font size="2">

- Herman Saffar, O. 2022. An Approach for Choosing Number of Clusters for K-Means. [www.medium.com](https://medium.com/towards-data-science/an-approach-for-choosing-number-of-clusters-for-k-means-c28e614ecb2c)

{% for i in project().rule_used['fastani']['references'] %}
- *{{ i }}*
{% endfor %}
</font>